In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

# Настройки
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 7)
sns.set_palette("husl")

OUTPUT_DIR = f'plots_analysis'
Path(OUTPUT_DIR).mkdir(exist_ok=True)

In [ ]:
# Функция для построения графиков
def plot_combined_methods(df, matrix_sizes):
    for size in matrix_sizes:
        plt.figure(figsize=(14, 8))
        size_data = df[df['size_num'] == size]
        
        # Группируем по комбинации метод+проекция и рангу
        avg_times = size_data.groupby(['method_proj', 'true_rank'])['time'].mean().reset_index()
        
        # Строим график для каждой комбинации
        for method in methods:
            method_data = avg_times[avg_times['method_proj'] == method]
            if not method_data.empty:
                plt.plot(method_data['true_rank'], method_data['time'], 
                        label=method, linewidth=2, marker='o', markersize=6)
        
        plt.title(f'Среднее время выполнения (Матрица {size}x{size})', fontsize=14, pad=20)
        plt.xlabel('Ранг', fontsize=12, labelpad=10)
        plt.ylabel('Время (секунды)', fontsize=12, labelpad=10)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        plt.grid(True, alpha=0.3)
        
        # Сохраняем
        filename = os.path.join(OUTPUT_DIR, f'time_vs_rank_{size}x{size}.png')
        plt.tight_layout()
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Сохранено: {filename}")

In [ ]:
m, n = 2**8, 2**8

In [ ]:
# Пути (проверьте!)
INPUT_CSV = f'test_{m}_{n}_results'
# Загрузка данных
try:
    df = pd.read_csv(INPUT_CSV)
    print(f"Успешно загружено {len(df)} записей")
except Exception as e:
    print(f"Ошибка: {e}")
    raise

In [ ]:
# Создаем комбинированный столбец "метод_проекция"
df['method_proj'] = df['method'] + '_' + df['projections'].astype(str)
methods = df['method_proj'].unique()

print("\nУникальные комбинации метод+проекция:")
print(methods)

# Извлекаем размер матрицы (исправленное регулярное выражение)
df['size_num'] = df['matrix_size'].str.extract(r'(\d+)').astype(int)
matrix_sizes = sorted(df['size_num'].unique())

In [ ]:
plot_combined_methods(df, matrix_sizes)

# Дополнительно: сводная таблица
print("\nСводная статистика по времени выполнения:")
stats_table = df.groupby(['method_proj', 'size_num', 'true_rank'])['time'].agg(['mean', 'std', 'count'])
display(stats_table.head(10))